<a href="https://colab.research.google.com/github/wheemin-2/25-1-ESAA/blob/main/0502_HW_Recommendations_Practice2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **행렬 분해를 이용한 잠재 요인 협업 필터링 실습**

(Review)

행렬 분해 잠재 요인 협업 필터링은 SVD나 NMF 등을 적용할 수 있는데, 일반적으로 행렬 분해에는 SVD가 자주 사용됨. 하지만 사용자-아이템 평점 행렬에는 사용자가 평점을 매기지 않은 Null 데이터가 많기 때문에 주로 SGD나 ALS 기반의 행렬 분해를 이용

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd

movies = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ESAA/25-1 OB/movies.csv')
ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ESAA/25-1 OB/ratings.csv')

In [ ]:
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두 개의 분해된 행렬 P와 Q.T의 내적으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)

    # 실제 R 행렬에서 NULL이 아닌 값의 위치 인덱스를 추출해 실제 R 행렬과 R_hat의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)

    return rmse

In [ ]:
def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda=0.01):
    num_users, num_items = R.shape
    # P와 Q 매트릭스의 크기 지정, 정규 분포를 가진 랜덤한 값으로 입력
    np.random.seed(1)
    P = np.random.normal(scale=1./K, size=(num_users, K))
    Q = np.random.normal(scale=1./K, size=(num_items, K))

    # R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장
    non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0]

    # SGD 기법으로 P와 Q 매트릭스를 계속해서 업데이트
    for step in range(steps):
        for i, j, r in non_zeros:
            # 실제값과 예측값의 차이인 오류값 구함
            eij = r - np.dot(P[i,:], Q[j,:].T)
            # Regularization을 반영한 SGD 업데이트 공식 적용
            P[i, :] = P[i, :] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
            Q[j, :] = Q[j, :] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])

        rmse = get_rmse(R, P, Q, non_zeros)
        if (step % 10)==0:
            print('### iteration step:', step, 'rmse:', rmse)

    return P, Q

In [ ]:
ratings = ratings[['userId','movieId','rating']]
ratings_matrix = ratings.pivot_table('rating',index='userId',columns='movieId')

# title 칼럼을 얻기 위해 movies와 조인 수행
rating_movies = pd.merge(ratings, movies, on='movieId')

# title 칼럼으로 pivot 수행
ratings_matrix = rating_movies.pivot_table('rating',index='userId',columns='title')

In [ ]:
# 사용자-아이템 평점 행렬을 분해
# 수행 시간이 오래 걸리므로 SGD 반복 횟수인 steps=200으로 설정
# 잠재 요인 차원 K = 50, L2 규제 계수는 0.01로 설정
P, Q = matrix_factorization(ratings_matrix.values, K=50, steps=200,
                            learning_rate=0.01, r_lambda=0.01)
pred_matrix = np.dot(P, Q.T)

### iteration step: 0 rmse: 2.9023619751336867
### iteration step: 10 rmse: 0.7335768591017927
### iteration step: 20 rmse: 0.5115539026853442
### iteration step: 30 rmse: 0.37261628282537446
### iteration step: 40 rmse: 0.2960818299181014
### iteration step: 50 rmse: 0.2520353192341642
### iteration step: 60 rmse: 0.22487503275269854
### iteration step: 70 rmse: 0.2068545530233154
### iteration step: 80 rmse: 0.19413418783028685
### iteration step: 90 rmse: 0.18470082002720406
### iteration step: 100 rmse: 0.17742927527209104
### iteration step: 110 rmse: 0.1716522696470749
### iteration step: 120 rmse: 0.16695181946871726
### iteration step: 130 rmse: 0.16305292191997542
### iteration step: 140 rmse: 0.15976691929679646
### iteration step: 150 rmse: 0.1569598699945732
### iteration step: 160 rmse: 0.15453398186715425
### iteration step: 170 rmse: 0.15241618551077643
### iteration step: 180 rmse: 0.1505508073962831
### iteration step: 190 rmse: 0.1488947091323209


In [ ]:
ratings_pred_matrix = pd.DataFrame(data=pred_matrix, index=ratings_matrix.index,
                                   columns=ratings_matrix.columns)
ratings_pred_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,3.055084,4.092018,3.564130,4.502167,3.981215,1.271694,3.603274,2.333266,5.091749,3.972454,...,1.402608,4.208382,3.705957,2.720514,2.787331,3.475076,3.253458,2.161087,4.010495,0.859474
2,3.170119,3.657992,3.308707,4.166521,4.311890,1.275469,4.237972,1.900366,3.392859,3.647421,...,0.973811,3.528264,3.361532,2.672535,2.404456,4.232789,2.911602,1.634576,4.135735,0.725684
3,2.307073,1.658853,1.443538,2.208859,2.229486,0.780760,1.997043,0.924908,2.970700,2.551446,...,0.520354,1.709494,2.281596,1.782833,1.635173,1.323276,2.887580,1.042618,2.293890,0.396941


**userId = 9 인 사람에 대한 영화 추천 알고리즘**

In [ ]:
# 사용자가 평점을 주지 않은 영화를 리스트로 반환하는 함수 생성
def get_unseen_movies(ratings_matrix, userId):
    user_rating = ratings_matrix.loc[userId, :]
    # 사용자가 이미 관람한 영화 리스트
    already_seen = user_rating[user_rating > 0].index.tolist()
    # 모든 영화명을 리스트 객체로 생성
    movies_list = ratings_matrix.columns.tolist()
    # list comprehension으로 아직 관람하지 않은 영화 리스트 생성
    unseen_list = [ movie for movie in movies_list if movie not in already_seen]

    return unseen_list

In [ ]:
def recomm_movie_by_user_id(pred_df, userId, unseen_list, top_n=10):
    # 예측 평점 DataFrame에서 사용자id와 unseen list로 들어온 영화명 칼럼을 추출해
    # 가장 예측 평점이 높은 순으로 정렬
    recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies

# 사용자가 관람하지 않은 영화명 추출
unseen_list = get_unseen_movies(ratings_matrix, 9)

# 아이템 기반의 최근접 이웃 협업 필터링으로 영화 추천
recomm_movies = recomm_movie_by_user_id(ratings_pred_matrix, 9, unseen_list, top_n=10)

# 평점 데이터를 DF로 생성
recomm_movies = pd.DataFrame(data=recomm_movies.values, index=recomm_movies.index,
                             columns=['pred_score'])

recomm_movies


,pred_score
title,
Rear Window (1954),5.704612
"South Park: Bigger, Longer and Uncut (1999)",5.451100
Rounders (1998),5.298393
Blade Runner (1982),5.244951
Roger & Me (1989),5.191962
Gattaca (1997),5.183179
Ben-Hur (1959),5.130463
Rosencrantz and Guildenstern Are Dead (1990),5.087375
"Big Lebowski, The (1998)",5.038690


In [ ]:
# 사용자가 관람하지 않은 영화명 추출
unseen_list = get_unseen_movies(ratings_matrix, 9)

# 잠재 요인 협업 필터링으로 영화 추천
recomm_movies = recomm_movie_by_user_id(ratings_pred_matrix, 9, unseen_list, top_n=10)

# 평점 데이터를 DataFrame으로 생성
recomm_movies = pd.DataFrame(data=recomm_movies.values, index=recomm_movies.index,
                             columns=['pred_score'])
recomm_movies

,pred_score
title,
Rear Window (1954),5.704612
"South Park: Bigger, Longer and Uncut (1999)",5.451100
Rounders (1998),5.298393
Blade Runner (1982),5.244951
Roger & Me (1989),5.191962
Gattaca (1997),5.183179
Ben-Hur (1959),5.130463
Rosencrantz and Guildenstern Are Dead (1990),5.087375
"Big Lebowski, The (1998)",5.038690


# **파이썬 추천 시스템 패키지 - Surprise**

## **Surprise 패키지 소개**

Surprise 패키지는 API를 이용해 쉽게 추천 시스템을 구축할 수 있게 만들어짐

주요 장점
- 다양한 추천 알고리즘, 예를 들어 사용자 또는 아이템 기반 최근접 이웃 협업 필터링, SVD, SVD++, NMF 기반의 잠재요인 협업 필터링을 쉽게 적용해 추천 시스템 구축 가능
- 사이킷런과 유사한 API명 사용
    - `fit()`, `predict()`, `train_test_split()`, `cross_validate()` 등 사용

## **Surprise를 이용한 추천 시스템 구축**

In [2]:
# 넘파이 버전 다운그레이드
!pip install 'numpy<2'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 68.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [3]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 8.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505215 sha256=5b36ed37b261e11310b97cf10e6362de97ee5e7a4ed99fca9da5143722b9b017
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [4]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [5]:
# Surprise에서 데이터 로딩은 Dataset 클래스를 이용해서만 가능
# load_builtin : 무비렌즈 사이트에서 제공하는 과거 버전 데이터 세트
# ml-100k : 10만 개 평점 데이터, ml-1m : 백만 개 평점 데이터
data = Dataset.load_builtin('ml-100k')

trainset, testset = train_test_split(data, test_size=.25, random_state=0)

In [6]:
# surprise는 자체적으로 로우 레벨의 데이터를 칼럼 레벨의 데이터로 변경
# 원본 데이터 세트를 그대로 사용
algo = SVD(random_state=0)
algo.fit(trainset)

In [7]:
# test() : 사용자-아이템 평점 데이터 세트 전체에 대해서 추천을 예측
# predict() : 개별 사용자와 영화에 대한 추천 평점 반환
predictions = algo.test(testset)
print('prediction type :', type(predictions), 'size:', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'> size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.5114147666251547, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.573872419581491, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.033583485472447, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.8463639495936905, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.1807542478219157, details={'was_impossible': False})]

`test()` 메서드의 호출 결과 설명
- type : 파이썬 리스트
- size : 입력 인자 데이터 세트의 크기와 동일 (25,000개)
- 개별 사용자 아이디(uid), 영화(아이템) 아이디(iid), 실제 평점(r_ui) 정보에 기반하여 추천 예측 평점(est) 데이터를 가짐
- details 속성 : 내부 처리 시 추천 예측을 알 수 없는 경우에 로그용으로 데이터를 남기는 데 사용됨
    - 'was_impossible'=True 이면 예측값을 생성할 수 없는 데이터라는 의미

In [8]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.5114147666251547),
 ('882', '291', 3.573872419581491),
 ('535', '507', 4.033583485472447)]

In [9]:
# predict() 메서드 사용법
# 인자로 개별 사용자 아이디, 아이템 아이디 입력 시 추천 예측 평점을 반환
# 입력 인자는 반드시 문자열!
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.49   {'was_impossible': False}


In [10]:
# 예측 평점과 실제 평점의 차이 평가 : Surprise의 accuracy 모듈 사용
accuracy.rmse(predictions)

RMSE: 0.9467


0.9466860806937948

## **Surprise 주요 모듈 소개**

### **Dataset**

- surprise는 'user_id'(사용자 아이디), item_id(아이템 아이디), rating(평점) 데이터가 **로우 레벨로 된** 데이터 셋만 적용 가능

- 만약 user_id, item_id, rating, time_stamp로 구성된 데이터가 있다면 앞 3개 컬럼만 로딩함.

- 일반 데이터 파일이나 판다스 데이터 프레임도 로딩 가능하나 컬럼 순서가 반드시 사용자 아이디, 아이템 아이디, 평점 순이어야 함!!
    - Surprise는 순서가 맞춰져 있다고 가정하고 네 번째 칼럼부터는 아예 로딩을 수행하지 않기 때문

[주요 모듈 소개]

- `load_builtin(name='ml-100k)` : 무비렌즈 데이터를 불러올 수 있음
    - 디폴트로 ml-100k(10만 개 평점 데이터)를 불러오며 ml-1M(100만 개 평점 데이터)를 불러 올 수 있음
- `load_from_file(file_path, reader)` : OS 파일에서 데이터를 로딩할 때 사용
    - 콤마(,), tab으로 칼럼이 분리된 포맷의 OS 파일에서 데이터를 로딩

- `load_from_df(df, reader)` : 판다스의 DataFrame에서 데이터를 로딩
    - df 는 사용자 아이디, 아이템 아이디, 평점 순으로 칼럼 순서가 정해져 있어야 함



**OS 파일 데이터를 Surprise 데이터 세트로 로딩**

Surprise에 OS 파일을 로딩할 때 주의할 점 : 로딩되는 데이터 파일에 칼럼명을 가지는 헤더 문자열이 있어서는 안 됨!!
- 칼럼 Header 제거가 필요함

Surprise 데이터 세트는 기본적으로 무비렌즈 데이터 형식을 따름
- 무비렌즈 데이터 형식이 아닌 다른 OS 파일의 경우 Reader 클래스를 먼저 설정해야 함



```
Reader(line_format, sep, rating_scale)
```
- line_format (string) : 칼럼을 순서대로 나열, 입력된 문자열을 공백으로 분리해 칼럼으로 인식함
- sep : 칼럼을 분리하는 문자, 판다스 DataFrame에서 입력받을 경우 기재할 필요 없음
    - '\t'가 디폴트
- rating_scale (tuple, optional) : 평점값의 최소 ~ 최대 평점 설정
    - (1,5)가 디폴트


In [11]:
import pandas as pd

ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ESAA/25-1 OB/ratings.csv')
# 인덱스, 헤더 모두 제외한 파일 생성
ratings.to_csv('ratings_noh.csv', index=False, header=False)

In [12]:
from surprise import Reader

# Reader 클래스를 이용해 데이터 파일의 파싱 포맷을 정의
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5,5))
data=Dataset.load_from_file('/content/ratings_noh.csv',reader=reader) # 앞의 3개 칼럼만 로딩됨

In [13]:
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습하고 나서 테스트 데이터 세트로 평점 예측 후 평가
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

**판다스 DataFrame에서 Surprise 데이터 세트로 로딩**

`Dataset.load_from_df()`를 이용하면 판다스의 DataFrame에서도 Surpise 데이터 세트로 로딩 가능
- 주의할 점 : 칼럼 순서를 지켜야 함 (사용자 아이디, 아이템 아이디, 평점 순)

In [14]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ESAA/25-1 OB/ratings.csv')
reader = Reader(rating_scale=(0.5, 5.0))

# 칼럼 순서 맞추기
data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

## **Surprise 추천 알고리즘 클래스**

![surprise](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fs5nS7%2Fbtq5oqzAakj%2FnkFnrsWuDkCX8NZzk9xKfk%2Fimg.png)

Surprise SVD의 비용 함수 : 사용자 베이스라인 편향성을 감안한 평점 예측에 Regularization을 적용한 것

[SVD 클래스 입력 파라미터]
- n_factors : 잠재 요인 K의 개수
    - 100이 디폴트
    - 커질수록 정확도가 높아질 수는 있으나 과적합 문제 발생 가능
- n_epochs : SGD 수행 시 반복 횟수
    - 20이 디폴트
- biased (bool) : 베이스라인 사용자 편향 적용 여부 (True가 디폴트)

## **베이스라인 평점**

베이스라인 평점(Baseline Rating)

- 개인의 성향을 반영해 아이템 평가에 편향성(bias) 요소를 반영하여 평점을 부과하는 것
- 보통 전체 평균 평점 + 사용자 편향 점수 + 아이템 편향 점수 공식으로 계산됨
    - 전체 평균 평점 : 모든 사용자의 아이템에 대한 평점을 평균한 값
    - 사용자 편향 점수 : 사용자별 아이템 평점 평균 값 - 전체 평균 평점
    - 아이템 편향 점수 : 아이템별 평점 평균 값 - 전체 평균 평점

![baseline](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FmKSkA%2Fbtq5kZvTtdW%2Fpkm4aU89KdkEnkBNSjzNoK%2Fimg.png)

## **교차 검증과 하이퍼 파라미터 튜닝**

Surprise는 교차 검증과 하이퍼 파라미터 튜닝을 위해 사이킷런과 유사한 `cross_validate( )`와 `GridSearchCV` 클래스를 제공함

In [15]:
from surprise.model_selection import cross_validate
from surprise import Reader

# 판다스 DataFrame에서 Surprise 데이터 세트로 로딩
ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ESAA/25-1 OB/ratings.csv')
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE','MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8791  0.8660  0.8801  0.8792  0.8679  0.8744  0.0062  
MAE (testset)     0.6768  0.6654  0.6763  0.6760  0.6668  0.6723  0.0050  
Fit time          2.90    1.89    2.27    1.41    1.49    1.99    0.55    
Test time         0.23    0.17    0.11    0.10    0.14    0.15    0.05    


{'test_rmse': array([0.87910334, 0.86596307, 0.88006198, 0.8791724 , 0.86791436]),
 'test_mae': array([0.67675128, 0.66540437, 0.67632683, 0.67599688, 0.66683188]),
 'fit_time': (2.900792121887207,
  1.8940305709838867,
  2.26613450050354,
  1.4081368446350098,
  1.4898521900177002),
 'test_time': (0.22984623908996582,
  0.17318940162658691,
  0.11163926124572754,
  0.1018829345703125,
  0.1359539031982422)}

In [16]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터를 딕셔너리 형태로 지정
param_grid = {'n_epochs':[20,40,60], 'n_factors':[50,100,200]}

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse로 수행하도록 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse','mae'], cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그 때의 하이퍼 파라미터 출력
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8762781184340706
{'n_epochs': 20, 'n_factors': 50}


## **Surprise를 이용한 개인화 영화 추천 시스템 구축**

In [18]:
# train_test_split()으로 분리되지 않는 데이터 세트에 fit()을 호출하면 오류 발생
data=Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)
algo = SVD(n_factors=50, random_state=0)
algo.fit(data)

AttributeError: 'DatasetAutoFolds' object has no attribute 'n_users'

In [19]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5,5))
data_folds = DatasetAutoFolds(ratings_file='/content/ratings_noh.csv', reader=reader)

# 전체 데이터를 학습 데이터로 생성
trainset = data_folds.build_full_trainset()

In [25]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [26]:
# 영화에 대한 상세 속성 정보 DataFrame 로딩
movies = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ESAA/25-1 OB/movies.csv')

# userId=9의 movieId 데이터를 추출해 movieId=42 데이터가 있는지 확인
movieIds = ratings[ratings['userId']==9]['movieId']

if movieIds[movieIds==42].count() ==0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [27]:
# predict() 메서드를 이용하여 특정 사용자의 예상 평점 계산
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [29]:
# 사용자가 평점을 매기지 않은 전체 영화에 대한 예측 평점 계산
def get_unseen_surprise(ratings, movies, userId):
    seen_movies = ratings[ratings['userId']==userId]['movieId'].tolist()
    total_movies = movies['movieId'].tolist()

    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화 수:', len(seen_movies), '추천 대상 영화 수:', len(unseen_movies),
          '전체 영화 수:', len(total_movies))

    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742


In [30]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    # 알고리즘 객체의 predict 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

    # predictions list 객체는 surprise의 Prediction 객체를 원소로 가짐
    # 이를 est 값으로 정렬하기 위해 sortkey_est 함수 정의
    def sortkey_est(pred):
        return pred.est

    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions = predictions[:top_n]

    # top_n으로 추출된 영화의 정보 추출
    top_movie_ids = [int(pred.iid) for pred in top_predictions]
    top_movie_rating = [pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']

    top_movie_preds = [(id, title, rating) for id, title, rating in
                       zip(top_movie_ids, top_movie_titles, top_movie_rating)]

    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

print('#### Top-10 추천 영화 리스트 ####')
for top_movie in top_movie_preds:
    print(top_movie[1], ':', top_movie[2])

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742
#### Top-10 추천 영화 리스트 ####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122657
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
